In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
tf.config.optimizer.set_jit(False)

In [3]:
# Step 1: Load Dataset
# -----------------------------------------------------
train_ds, test_ds = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"],
    as_supervised=True   # returns (image, label)
)

In [4]:
# Step 2: Preprocessing Function
# -----------------------------------------------------
NUM_CLASSES = 5
IMG_SIZE = (150, 150)

In [5]:
def preprocess(image, label):
    # Resize
    image = tf.image.resize(image, IMG_SIZE)
    # Preprocess for VGG16
    image = preprocess_input(image)
    # One-hot encode labels
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

In [6]:
# Apply preprocessing + batching
train_ds = (train_ds
            .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
            .shuffle(1000)
            .batch(32)
            .prefetch(tf.data.AUTOTUNE))

In [7]:
test_ds = (test_ds
           .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(32)
           .prefetch(tf.data.AUTOTUNE))

In [8]:
# Step 3: Load Pre-trained Model (VGG16)
# -----------------------------------------------------
base_model = VGG16(weights="imagenet",
                   include_top=False,
                   input_shape=(150, 150, 3))

In [9]:
base_model.trainable = False  # freeze convolutional base

In [10]:
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 150, 150, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 150, 150, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 75, 75, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 75, 75, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 75, 75, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 37, 37, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 37, 37, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 18, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 18, 18, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 9, 9, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 4, 4, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [11]:
# Custom classifier
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(50, activation='relu'),
    layers.Dense(20, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 4, 4, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 50)                  │         409,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │           1,020 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │             105 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,125,463 (57.70 MB)

 Trainable params: 410,775 (1.57 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [13]:
# Step 4: Compile & Train
# -----------------------------------------------------
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_accuracy',
                   mode='max',
                   patience=5,
                   restore_best_weights=True)

In [14]:
history = model.fit(
    train_ds,
    epochs=5,
    validation_data=test_ds,
    callbacks=[es]
)

Epoch 1/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 855s 11s/step - accuracy: 0.5944 - loss: 1.4571 - val_accuracy: 0.7302 - val_loss: 0.8007
Epoch 2/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 896s 11s/step - accuracy: 0.7953 - loss: 0.6784 - val_accuracy: 0.8774 - val_loss: 0.4318
Epoch 3/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 790s 10s/step - accuracy: 0.8723 - loss: 0.4035 - val_accuracy: 0.9192 - val_loss: 0.3105
Epoch 4/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 765s 9s/step - accuracy: 0.9074 - loss: 0.2943 - val_accuracy: 0.9482 - val_loss: 0.1629
Epoch 5/5
81/81 ━━━━━━━━━━━━━━━━━━━━ 820s 10s/step - accuracy: 0.9397 - loss: 0.1931 - val_accuracy: 0.9609 - val_loss: 0.1151


In [15]:
# -----------------------------------------------------
loss, accuracy = model.evaluate(test_ds)
print("Loss:", loss, "Accuracy:", accuracy)

35/35 ━━━━━━━━━━━━━━━━━━━━ 337s 10s/step - accuracy: 0.9609 - loss: 0.1151
Loss: 0.11514371633529663 Accuracy: 0.9609445929527283


In [ ]:
# Step 6: Plot Accuracy
# -----------------------------------------------------
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('Training Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'loss'], loc='upper left')
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

for image, label in test_ds.shuffle(1000).take(1):

    plt.imshow(image)
    plt.show()

    image = tf.image.resize(image, (150, 150))
    image = tf.keras.applications.vgg16.preprocess_input(image)

    img = tf.expand_dims(image, axis=0)
    pred = model.predict(img)
    pred_class = np.argmax(pred)

    if(pred_class == 0):
        print("Predicted image is: dandelion")
    elif(pred_class == 1):
        print("Predicted image is: daisy")
    elif(pred_class == 2):
        print("Predicted image is: tulips")
    elif(pred_class == 3):
        print("Predicted image is: sunflowers")
    elif(pred_class == 4):
        print("Predicted image is: roses")